In [5]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# Dataset yükləndi
df = pd.read_csv('/content/raw_data.csv')

# Unikal ölkələri tap
countries = df['Area'].unique()

# Nominatim obyektini yarat
geolocator = Nominatim(user_agent="crop_prediction_app_v2", timeout=10)

# Saniyədə maksimum 1 sorğu göndərmək üçün RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

country_coords = {}

print(f"{len(countries)} ölkə üçün koordinatlar tapılır...")

for country in countries:
    try:
        # Sorğu göndər
        location = geocode(country)
        if location:
            country_coords[country] = (location.latitude, location.longitude)
            print(f"Tapıldı: {country} -> {location.latitude}, {location.longitude}")
        else:
            country_coords[country] = (None, None)
            print(f"Tapılmadı: {country}")
    except Exception as e:
        print(f"Xəta baş verdi ({country}): {e}")
        country_coords[country] = (None, None)
        time.sleep(2) # Xəta olduqda fasile

# Koordinatları datasetə əlavə olunur
df['lat'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[0])
df['lon'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[1])

print("Əməliyyat tamamlandı!")

101 ölkə üçün koordinatlar tapılır...
Tapıldı: Albania -> 41.000028, 19.9999619
Tapıldı: Algeria -> 28.0000272, 2.9999825
Tapıldı: Angola -> -11.8775768, 17.5691241
Tapıldı: Argentina -> -34.9964963, -64.9672817
Tapıldı: Armenia -> 40.7696272, 44.6736646
Tapıldı: Australia -> -24.7761086, 134.755
Tapıldı: Austria -> 47.59397, 14.12456
Tapıldı: Azerbaijan -> 40.3936294, 47.7872508
Tapıldı: Bahamas -> 24.7736546, -78.0000547
Tapıldı: Bahrain -> 26.1551249, 50.5344606
Tapıldı: Bangladesh -> 24.4769288, 90.2934413
Tapıldı: Belarus -> 53.4250605, 27.6971358
Tapıldı: Belgium -> 50.6402809, 4.6667145
Tapıldı: Botswana -> -23.1681782, 24.5928742
Tapıldı: Brazil -> -10.3333333, -53.2
Tapıldı: Bulgaria -> 42.6073975, 25.4856617
Tapıldı: Burkina Faso -> 12.0753083, -1.6880314
Tapıldı: Burundi -> -3.426449, 29.9324519
Tapıldı: Cameroon -> 4.6125522, 13.1535811
Tapıldı: Canada -> 61.0666922, -107.991707
Tapıldı: Central African Republic -> 7.0323598, 19.9981227
Tapıldı: Chile -> -31.7613365, -71.31

Tapıldı: Jamaica -> 18.1850507, -77.3947693
Tapıldı: Japan -> 36.5748441, 139.2394179
Tapıldı: Kazakhstan -> 48.1012954, 66.7780818
Tapıldı: Kenya -> 1.4419683, 38.4313975
Tapıldı: Latvia -> 56.8406494, 24.7537645
Tapıldı: Lebanon -> 33.8750629, 35.843409
Tapıldı: Lesotho -> -29.6039267, 28.3350193
Tapıldı: Libya -> 26.8234472, 18.1236723
Tapıldı: Lithuania -> 55.3500003, 23.7499997
Tapıldı: Madagascar -> -18.9249604, 46.4416422
Tapıldı: Malawi -> -13.2687204, 33.9301963
Tapıldı: Malaysia -> 4.5693754, 102.2656823
Tapıldı: Mali -> 16.3700359, -2.2900239
Tapıldı: Mauritania -> 20.2540382, -9.2399263
Tapıldı: Mauritius -> -20.2759451, 57.5703566
Tapıldı: Mexico -> 23.6585116, -102.0077097
Tapıldı: Montenegro -> 42.7755624, 19.2022423
Tapıldı: Morocco -> 28.3347722, -10.3713379
Tapıldı: Mozambique -> -19.302233, 34.9144977
Tapıldı: Namibia -> -23.2335499, 17.3231107
Tapıldı: Nepal -> 28.3780464, 83.9999901
Tapıldı: Netherlands -> 52.2434979, 5.6343227
Tapıldı: New Zealand -> -41.5000831, 

Tapıldı: Ukraine -> 49.4871968, 31.2718321
Tapıldı: United Kingdom -> 54.7023545, -3.2765753
Tapıldı: Uruguay -> -32.8755548, -56.0201525
Tapıldı: Zambia -> -14.5189121, 27.5589884
Tapıldı: Zimbabwe -> -18.4554963, 29.7468414
Əməliyyat tamamlandı!


In [6]:
import requests
import pandas as pd
import time

# 1. Unikal Ölkə və İl cütlerini tap
weather_targets = df[['Area', 'Year', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)

def get_annual_temp(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None

    # Open-Meteo API URL
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={year}-01-01&end_date={year}-12-31&daily=temperature_2m_mean&timezone=auto"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Gündəlik ortalama temperaturlar siyahısı
            daily_temps = data['daily']['temperature_2m_mean']
            # İl üzrə ortalama temperatur
            if daily_temps:
                annual_avg = sum(daily_temps) / len(daily_temps)
                return round(annual_avg, 2)
        return None
    except:
        return None

# 2. Temperatur məlumatlarını çəkməyə başlayaq
print(f"Cəmi {len(weather_targets)} unikal sətir üçün temperatur çəkilir...")

temps = []
for index, row in weather_targets.iterrows():
    temp = get_annual_temp(row['lat'], row['lon'], row['Year'])
    temps.append(temp)

    # Prosesi izləmək üçün print edirik
    if index % 10 == 0:
        print(f"{index}/{len(weather_targets)} sətir tamamlandı: {row['Area']} ({row['Year']}) -> {temp}°C")

    # Fasilə
    time.sleep(0.05)

# 3. Nəticələr birlesdirilir
weather_targets['avg_temp'] = temps
df = pd.merge(df, weather_targets[['Area', 'Year', 'avg_temp']], on=['Area', 'Year'], how='left')

print("Bütün temperaturlar uğurla əlavə edildi!")
print(df.head())

Cəmi 2250 unikal sətir üçün temperatur çəkilir...
0/2250 sətir tamamlandı: Albania (1990) -> 14.99°C
10/2250 sətir tamamlandı: Albania (2000) -> 15.01°C
20/2250 sətir tamamlandı: Albania (2011) -> 15.13°C
30/2250 sətir tamamlandı: Algeria (1997) -> 24.19°C
40/2250 sətir tamamlandı: Algeria (2008) -> 24.25°C
50/2250 sətir tamamlandı: Angola (1994) -> 21.04°C
60/2250 sətir tamamlandı: Angola (2005) -> 21.53°C
70/2250 sətir tamamlandı: Argentina (1991) -> 16.39°C
80/2250 sətir tamamlandı: Argentina (2001) -> 16.77°C
90/2250 sətir tamamlandı: Argentina (2012) -> 17.22°C
100/2250 sətir tamamlandı: Armenia (2000) -> 2.18°C
110/2250 sətir tamamlandı: Armenia (2011) -> 1.39°C
120/2250 sətir tamamlandı: Australia (1997) -> 22.97°C
130/2250 sətir tamamlandı: Australia (2008) -> 23.34°C
140/2250 sətir tamamlandı: Austria (1994) -> 4.44°C
150/2250 sətir tamamlandı: Austria (2005) -> 2.87°C
160/2250 sətir tamamlandı: Azerbaijan (1993) -> 14.29°C
170/2250 sətir tamamlandı: Azerbaijan (2004) -> 16.15

In [7]:
print(df.columns)

Index(['Unnamed: 0', 'Area', 'Item', 'Year', 'hg/ha_yield',
       'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'lat', 'lon',
       'avg_temp'],
      dtype='object')


In [8]:
#  Dataseti yeni bir CSV faylı kimi yadda saxla
df.to_csv('crop_yield_with_weather.csv', index=False)

In [9]:
# Faylı Colab-dan öz kompüterinə yüklədin
from google.colab import files
files.download('crop_yield_with_weather.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>